In [3]:
import pandas as pd
import numpy as np
from get_data import get_yield
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import grangercausalitytests

# 1 year yield data
df_1 = get_yield(term=1)
# to weekly and take diff
df_1 = df_1.resample("W-FRI").last()
df_1 = df_1.diff().dropna()
# Skip last two months of data
df_1 = df_1.truncate(after = pd.to_datetime('2023-2-10'))


# corr = df_1.corr()

Y = df_1.copy()
Y = Y.sort_index()
Y = Y.apply(pd.to_numeric, errors="coerce")

df_1

,UK,CAN,DBR,FRN,SWISS,SEK,NOR,AUD,ITL,US,CNY
Start Date,,,,,,,,,,,
2016-02-26,0.002,0.033,0.042,-0.023,-0.138,0.0,-0.015,-0.012,-0.015,0.056,0.0
2016-03-04,0.026,0.031,-0.025,-0.027,0.011,0.0,-0.036,0.037,-0.058,0.086,-0.095
2016-03-11,0.076,0.007,0.021,0.043,0.096,0.0,-0.03,0.043,0.005,0.038,-0.041
2016-03-18,-0.064,-0.013,-0.006,-0.019,-0.149,0.0,-0.042,0.001,0.0,-0.085,-0.04
2016-03-25,-0.002,0.033,0.024,0.008,0.204,0.0,-0.053,0.023,0.031,0.0,-0.043
...,...,...,...,...,...,...,...,...,...,...,...
2023-01-13,0.122,-0.033,-0.076,0.017,0.0,0.025,0.0,-0.15,-0.012,-0.019,0.017
2023-01-20,0.152,-0.131,0.066,0.039,0.0,0.004,0.0,-0.17,-0.009,-0.004,0.0
2023-01-27,0.007,0.017,-0.061,-0.023,0.0,0.025,0.0,0.213,0.011,-0.011,0.0


In [4]:
# Granger causlity test (just for UK-CAN for now)
# Might need to make dict of dicts for all of the combinations
gc_test_result = grangercausalitytests(Y[['CAN', 'UK']], maxlag=5, addconst=True, verbose=True)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.8514  , p=0.3568  , df_denom=360, df_num=1
ssr based chi2 test:   chi2=0.8585  , p=0.3541  , df=1
likelihood ratio test: chi2=0.8575  , p=0.3544  , df=1
parameter F test:         F=0.8514  , p=0.3568  , df_denom=360, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=4.1554  , p=0.0164  , df_denom=357, df_num=2
ssr based chi2 test:   chi2=8.4272  , p=0.0148  , df=2
likelihood ratio test: chi2=8.3306  , p=0.0155  , df=2
parameter F test:         F=4.1554  , p=0.0164  , df_denom=357, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=2.7425  , p=0.0431  , df_denom=354, df_num=3
ssr based chi2 test:   chi2=8.3901  , p=0.0386  , df=3
likelihood ratio test: chi2=8.2941  , p=0.0403  , df=3
parameter F test:         F=2.7425  , p=0.0431  , df_denom=354, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=2.3834  , p=0.0511  

/Users/tarun/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/stattools.py:1488: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


In [5]:
# VAR
model = VAR(Y)
order = model.select_order(maxlags=7)
print(order.summary())
p = order.selected_orders["aic"]
results = model.fit(p)
print(results.summary())

 VAR Order Selection (* highlights the minimums) 
      AIC         BIC         FPE         HQIC   
-------------------------------------------------
0      -59.33     -59.21*   1.718e-26      -59.28
1      -59.95      -58.52   9.169e-27     -59.38*
2      -60.19      -57.44   7.283e-27      -59.09
3      -60.13      -56.07   7.706e-27      -58.52
4      -60.21      -54.83   7.204e-27      -58.07
5     -60.33*      -53.64  6.503e-27*      -57.66
6      -60.21      -52.21   7.429e-27      -57.03
7      -60.30      -50.98   7.005e-27      -56.60
-------------------------------------------------
  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Wed, 11, Feb, 2026
Time:                     15:35:59
--------------------------------------------------------------------
No. of Equations:         11.0000    BIC:                   -53.7014
Nobs:                     359.000    HQIC:                  -57.7150
Log likelihood:   

In [6]:
# Granger Causality of everything

# Statistical meaning, things with a low enough p value improve the prediction of of Y inside the VAR, eg: lags of CAN help
# predict the UK change in yield

for caused in results.names:
    for causing in results.names:
        if caused != causing:
            test = results.test_causality(caused, [causing], kind='f')
            print(f"{causing} -> {caused}: p-value = {test.pvalue}")

CAN -> UK: p-value = 0.0015776604837126622
DBR -> UK: p-value = 0.1811133053183779
FRN -> UK: p-value = 2.4461402924812456e-07
SWISS -> UK: p-value = 0.10224856917950097
SEK -> UK: p-value = 9.8315542315216e-06
NOR -> UK: p-value = 0.03260816964423804
AUD -> UK: p-value = 0.22241125989238184
ITL -> UK: p-value = 0.2689630591451703
US -> UK: p-value = 0.007058781349625261
CNY -> UK: p-value = 0.22407014285968596
UK -> CAN: p-value = 0.11123144772341689
DBR -> CAN: p-value = 0.16697423718298476
FRN -> CAN: p-value = 0.6452742741303628
SWISS -> CAN: p-value = 0.2533229572657949
SEK -> CAN: p-value = 0.014156278084234782
NOR -> CAN: p-value = 0.406207938432239
AUD -> CAN: p-value = 3.896942723379421e-06
ITL -> CAN: p-value = 0.01934095840890971
US -> CAN: p-value = 0.01787984910452305
CNY -> CAN: p-value = 0.5853802370891199
UK -> DBR: p-value = 0.0002767440758365075
CAN -> DBR: p-value = 0.21142827385920251
FRN -> DBR: p-value = 1.256570011070925e-07
SWISS -> DBR: p-value = 0.515443525343

In [7]:
fevd = results.fevd(10)
fevd.summary()

FEVD for UK
           UK       CAN       DBR       FRN     SWISS       SEK       NOR       AUD       ITL        US       CNY
0    1.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
1    0.868122  0.013891  0.004202  0.048970  0.000314  0.051220  0.000004  0.002809  0.000689  0.000036  0.009743
2    0.800665  0.031825  0.011730  0.077157  0.000501  0.054733  0.000059  0.010180  0.002337  0.001754  0.009059
3    0.736287  0.043428  0.010807  0.074997  0.024076  0.072015  0.011121  0.009922  0.004908  0.004064  0.008374
4    0.705437  0.042382  0.012839  0.070567  0.023453  0.073271  0.017797  0.009378  0.011432  0.025405  0.008039
5    0.651928  0.062984  0.021056  0.065378  0.021731  0.079406  0.019652  0.010137  0.018892  0.040813  0.008023
6    0.640496  0.068090  0.024097  0.064817  0.022549  0.078060  0.019313  0.013084  0.018536  0.042817  0.008142
7    0.630653  0.067099  0.028559  0.064847  0.024431  0.079061  0.019834  0

In [8]:
import numpy as np
import pandas as pd

def generalized_fevd(var_results, H=10, normalize=True):
    """
    Generalized FEVD (Pesaran-Shin) for a fitted statsmodels VARResults.
    
    Parameters
    ----------
    var_results : statsmodels.tsa.vector_ar.var_model.VARResults
        Fitted VAR results (e.g., `results = model.fit(p)`).
    H : int
        Forecast horizon in steps (e.g., weeks). Uses horizons 0..H-1 in sums.
    normalize : bool
        If True, row-normalize so contributions sum to 1 for each equation at each horizon.
        
    Returns
    -------
    fevd : np.ndarray
        Array of shape (H, n, n) where fevd[h, i, j] is contribution of shock j
        to variable i at horizon h (h=0..H-1). If normalize=True, each row sums to 1.
    names : list
        Variable names in order.
    """
    names = list(var_results.names)
    n = len(names)

    # Moving-average (MA) representation coefficients Psi_k
    # Psi has shape (H, n, n) with Psi[0] = I
    Psi = var_results.ma_rep(H)

    # Residual covariance matrix Σ (n x n)
    Sigma = np.asarray(var_results.sigma_u)

    # Precompute denominators: denom[h, i] = sum_{k=0}^{h} e_i' Psi_k Σ Psi_k' e_i
    # We'll build horizon-by-horizon contributions using cumulative sums.
    fevd = np.zeros((H, n, n), dtype=float)

    # For each horizon h, compute cumulative sums from k=0..h
    for h in range(H):
        denom = np.zeros(n, dtype=float)
        numer = np.zeros((n, n), dtype=float)

        for k in range(h + 1):
            A = Psi[k] @ Sigma  # (n x n)
            # denom_i adds (Psi_k Σ Psi_k')_ii
            denom += np.diag(A @ Psi[k].T)

            # numer_{i,j} adds (e_i' Psi_k Σ e_j)^2 / σ_jj
            # e_i' Psi_k Σ e_j is just A[i, j]
            numer += (A ** 2)

        # divide each column j by σ_jj (generalized shock scaling)
        sigma_diag = np.diag(Sigma).copy()
        # avoid division by 0 if any diag is 0 (shouldn't happen in sane VAR)
        sigma_diag[sigma_diag == 0] = np.nan

        numer = numer / sigma_diag  # broadcasts across rows

        # contribution at horizon h
        # θ_ij(h) = numer_ij / denom_i
        # (broadcast denom_i across columns)
        fevd[h] = numer / denom[:, None]

        if normalize:
            row_sums = fevd[h].sum(axis=1, keepdims=True)
            fevd[h] = fevd[h] / row_sums

    return fevd, names


def fevd_table(fevd, names, var, horizons=None):
    """
    Convenience: return a DataFrame for one dependent variable across horizons.
    var : str, dependent variable name.
    horizons : iterable of int or None -> use all.
    """
    idx = names.index(var)
    H = fevd.shape[0]
    if horizons is None:
        horizons = range(H)

    rows = []
    for h in horizons:
        rows.append(fevd[h, idx, :])

    df = pd.DataFrame(rows, columns=names, index=list(horizons))
    df.index.name = "horizon"
    return df


In [14]:
gfevd, names = generalized_fevd(results, H=10, normalize=True)
uk_g = fevd_table(gfevd, names, "UK")
can_g = fevd_table(gfevd, names, "CAN")
us_g = fevd_table(gfevd, names, "US")

uk_g


,UK,CAN,DBR,FRN,SWISS,SEK,NOR,AUD,ITL,US,CNY
horizon,,,,,,,,,,,
0,0.649367,0.061348,0.071506,0.034906,0.008560,0.001839,0.008568,0.083223,0.026707,0.053803,0.000172
1,0.601230,0.060395,0.066441,0.058631,0.008386,0.032316,0.008133,0.081237,0.024401,0.051422,0.007410
2,0.559349,0.068188,0.068835,0.084697,0.007816,0.033055,0.007636,0.083192,0.030190,0.050104,0.006938
3,0.531141,0.074530,0.065501,0.083464,0.022240,0.044138,0.016608,0.079002,0.029214,0.047576,0.006587
4,0.499129,0.073102,0.067013,0.078283,0.022872,0.045896,0.024386,0.077478,0.034853,0.070835,0.006155
5,0.481559,0.082943,0.070042,0.075952,0.022006,0.054125,0.025507,0.074577,0.036349,0.069803,0.007138
6,0.475839,0.087335,0.069906,0.075837,0.022289,0.053663,0.025165,0.077972,0.035852,0.068866,0.007274
7,0.467678,0.086038,0.072969,0.078759,0.022816,0.053964,0.025148,0.077011,0.039621,0.068710,0.007286
8,0.458906,0.086988,0.073312,0.079021,0.022742,0.054083,0.024703,0.075613,0.040839,0.076630,0.007163
